In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict

In [2]:
# Create a dataframe for Bandage Combined1
master_df= pd.read_csv('QueriesAndResultsMasterList.tsv', sep='\t')

In [3]:
#Drop the columns
master_df=master_df.drop(['BandageVSGraphAligner','BandageVSSPAligner','SPAlignerVSGraphAligner'],axis=1)

In [4]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,1,0,0.0,0,0,5151.0,24.0,884.0
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967.0,37.0,897.0
2,2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151.0,24.0,884.0
3,3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151.0,24.0,884.0
4,4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151.0,24.0,884.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967.0,37.0,897.0
374,374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967.0,37.0,894.0
375,375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967.0,37.0,894.0
376,376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967.0,37.0,894.0


In [5]:
#Remove the + or - from the Path_SPAligner
master_df['Path_SPAligner']=master_df['Path_SPAligner'].str.replace(r'[+-]', '', regex=True)

## This section compares the result from SPAligner and Bandage

In [6]:
#Convert Path_GraphAligner, Start_GraphAligner and End_GraphAligner into int
master_df= master_df.fillna(000)
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(int)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(int)
master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(int)
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,1,0,0.0,0,0,5151,24,884
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897,861.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,897
2,2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,24,884
3,3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,24,884
4,4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,24,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897,861.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,897
374,374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,894
375,375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,894
376,376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,894


In [7]:
master_df['Start_SPAligner']=master_df['Start_SPAligner'].astype(str)
master_df['End_SPAligner']=master_df['End_SPAligner'].astype(str)
master_df['Path_SPAligner']=master_df['Path_SPAligner'].astype(str)

In [8]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,1,0,0.0,0,0,5151,24,884
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897,861.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,897
2,2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,24,884
3,3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,24,884
4,4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,24,884,861.0,5151,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,24,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897,861.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,897
374,374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,894
375,375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,894
376,376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,894,858.0,5967,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,37,894


In [9]:
def compare_values_Bandage_SPAligner(row):
    if row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] and row['Extracted_Path'] == row['Path_SPAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_SPAligner']:
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_SPAligner'] and row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] :
        return 'MatchOnPathAndStart'
    else:
        return 'Different'

In [10]:
master_df['BandageVSSPAligner'] = master_df.apply(compare_values_Bandage_SPAligner, axis=1)

In [11]:
master_df.dtypes

Unnamed: 0              int64
Query                  object
Sequence_Query         object
Length_Bandage          int64
Path_Bandage           object
Start_Bandage          object
End_Bandage            object
Extracted_Path         object
Sequence_Bandage       object
Start_SPAligner        object
End_SPAligner          object
Length_SPAligner      float64
Path_SPAligner         object
Sequence_SPAligner     object
Path_GraphAligner       int64
Start_GraphAligner      int64
End_GraphAligner        int64
BandageVSSPAligner     object
dtype: object

In [12]:
master_df.to_csv("TestRe.tsv",sep='\t')

In [13]:
BandagevsSPAligner_df=master_df[['Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_SPAligner','End_SPAligner','Length_SPAligner','Path_SPAligner']]

In [14]:
BandagevsSPAligner_df

,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner
0,(24) 5151- (884),24,884,5151,1,0,0.0,0
1,(37) 5967+ (897),37,897,5967,37,897,861.0,5967
2,(24) 5151- (884),24,884,5151,24,884,861.0,5151
3,(24) 5151- (884),24,884,5151,24,884,861.0,5151
4,(24) 5151- (884),24,884,5151,24,884,861.0,5151
...,...,...,...,...,...,...,...,...
373,(37) 5967+ (897),37,897,5967,37,897,861.0,5967
374,(37) 5967+ (894),37,894,5967,37,894,858.0,5967
375,(37) 5967+ (894),37,894,5967,37,894,858.0,5967
376,(37) 5967+ (894),37,894,5967,37,894,858.0,5967


In [15]:
BandagevsSPAligner_df['BandageVSSPAligner'] = BandagevsSPAligner_df.apply(compare_values_Bandage_SPAligner, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1541/1140914095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['BandageVSSPAligner'] = BandagevsSPAligner_df.apply(compare_values_Bandage_SPAligner, axis=1)


In [16]:
BandagevsSPAligner_df.to_csv('TestRes.tsv',sep='\t')

In [17]:
BandagevsSPAligner_df.to_csv('TestRes2.tsv',sep='\t')

In [18]:
output_list = []

for index, row in BandagevsSPAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = row['Path_SPAligner']
    start_value= row['Start_SPAligner']
    end_value= row['Start_SPAligner']
    
    if path_value in path_values and start_value in start_values and end_value in end_values:
        output_list.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value  in end_values:
        output_list.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list.append('SinglePathMatch')
    else:
         output_list.append('Different')
    

BandagevsSPAligner_df['ResultsBandageVSSPAligner'] = output_list

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1541/2242201832.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['ResultsBandageVSSPAligner'] = output_list


In [19]:
def determine_final_result(row):
    if row['BandageVSSPAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSSPAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSSPAligner']

In [20]:
BandagevsSPAligner_df.dtypes

Path_Bandage                  object
Start_Bandage                 object
End_Bandage                   object
Extracted_Path                object
Start_SPAligner               object
End_SPAligner                 object
Length_SPAligner             float64
Path_SPAligner                object
BandageVSSPAligner            object
ResultsBandageVSSPAligner     object
dtype: object

In [21]:
BandagevsSPAligner_df['FinalResultBandageVSSPAligner'] = BandagevsSPAligner_df.apply(determine_final_result, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1541/761307203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['FinalResultBandageVSSPAligner'] = BandagevsSPAligner_df.apply(determine_final_result, axis=1)


In [22]:
BandagevsSPAligner_df.to_csv('TestResultCategoriesBandageSPAligner.tsv',sep='\t')

## This section compares the values from GraphAligner and Bandage

In [23]:
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(str)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(str)
master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(str)

In [24]:
BandagevsGraphAligner_df=master_df[['Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner']]

In [25]:
BandagevsGraphAligner_df.dtypes


Path_Bandage          object
Start_Bandage         object
End_Bandage           object
Extracted_Path        object
Start_GraphAligner    object
End_GraphAligner      object
Path_GraphAligner     object
dtype: object

In [26]:
def compare_values_Bandage_GraphAligner(row):
    if row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] and row['Extracted_Path'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_GraphAligner']:
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_GraphAligner'] and row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    else:
        return 'Different'

In [27]:
BandagevsGraphAligner_df['BandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(compare_values_Bandage_GraphAligner, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1541/607607642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['BandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(compare_values_Bandage_GraphAligner, axis=1)


In [28]:
BandagevsGraphAligner_df.to_csv('rESULT44.tsv',sep='\t')

In [29]:
output_list1 = []

for index, row in BandagevsGraphAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = row['Path_GraphAligner']
    start_value= row['Start_GraphAligner']
    end_value= row['End_GraphAligner']
    
    if path_value in path_values and start_value in start_values and end_value in end_values:
        output_list1.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list1.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value  in end_values:
        output_list1.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list1.append('SinglePathMatch')
    else:
         output_list1.append('Different')
    

BandagevsGraphAligner_df['ResultsBandageVSGraphAligner'] = output_list1

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1541/1820273756.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['ResultsBandageVSGraphAligner'] = output_list1


In [30]:
BandagevsGraphAligner_df.to_csv('rESULT445.tsv',sep='\t')

In [31]:
def determine_final_result_1(row):
    if row['BandageVSGraphAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSGraphAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSGraphAligner']

In [32]:
BandagevsGraphAligner_df['FinalResultBandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(determine_final_result_1, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1541/2292256490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['FinalResultBandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(determine_final_result_1, axis=1)


In [33]:
BandagevsGraphAligner_df.to_csv('TestResultCategoriesBandageGraphAligner.tsv',sep='\t')

## This section compares the results from GraphAligner and SPAligner

In [34]:
SPAlignervsGraphAligner_df=master_df[['Start_SPAligner','End_SPAligner','Path_SPAligner',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner']]

In [35]:
SPAlignervsGraphAligner_df.dtypes

Start_SPAligner       object
End_SPAligner         object
Path_SPAligner        object
Start_GraphAligner    object
End_GraphAligner      object
Path_GraphAligner     object
dtype: object

In [36]:
def compare_values_GraphAligner_SPAligner(row):
    if row['Start_SPAligner'] == row['Start_GraphAligner'] and row['End_SPAligner'] == row['End_GraphAligner'] and row['Path_SPAligner'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Path_SPAligner'] == row['Path_GraphAligner']:
        return 'MatchOnPath'
    elif row['Path_SPAligner'] == row['Path_GraphAligner'] and row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    else:
        return 'Different'

In [37]:
SPAlignervsGraphAligner_df['SPAlignerVSGraphAligner'] = SPAlignervsGraphAligner_df.apply(compare_values_GraphAligner_SPAligner, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1541/33428813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAlignervsGraphAligner_df['SPAlignerVSGraphAligner'] = SPAlignervsGraphAligner_df.apply(compare_values_GraphAligner_SPAligner, axis=1)


In [38]:
SPAlignervsGraphAligner_df.to_csv('TestResultCategoriesGraphAlignerSPAligner.tsv',sep='\t')